# 경찰청_경찰관서 위치 주소 현황(공공데이터포털)

In [1]:
import pandas as pd
import numpy as np
from functools import reduce

In [2]:
# police 데이터 불러오기 
police = pd.read_csv("C:/sooeun/DATAexam/ConvergenceProject-School_Violence/Data/경찰청_경찰관서위치정보(지구대,파출소포함)_20210924.csv", encoding='cp949')
police

,지방청,경찰서,관서명,구분,주소
0,서울청,서울중부,을지,지구대,서울특별시 중구 을지로 234
1,서울청,서울중부,광희,지구대,서울특별시 중구 퇴계로 375-1
2,서울청,서울중부,약수,지구대,"서울특별시 중구 동호로 5길 15, 약수지구대"
3,서울청,서울중부,신당,파출소,"서울특별시 중구 다산로 248 (신당동, 신당파출소)"
4,서울청,서울중부,장충,파출소,서울특별시 중구 동호로 261
...,...,...,...,...,...
2026,제주청,서귀포,안덕,파출소,제주특별자치도 서귀포시 안덕면 화순로 116
2027,제주청,서귀포,대정,파출소,제주특별자치도 서귀포시 대정읍 상모로 310
2028,제주청,서귀포,중문,파출소,제주특별자치도 서귀포시 천제연로 165
2029,제주청,서귀포,대신,파출소,제주특별자치도 서귀포시 신서귀로51번길 18


In [3]:
# 서울특별시 
police_seoul = police[police['지방청'].str.contains("서울")]
police_seoul

,지방청,경찰서,관서명,구분,주소
0,서울청,서울중부,을지,지구대,서울특별시 중구 을지로 234
1,서울청,서울중부,광희,지구대,서울특별시 중구 퇴계로 375-1
2,서울청,서울중부,약수,지구대,"서울특별시 중구 동호로 5길 15, 약수지구대"
3,서울청,서울중부,신당,파출소,"서울특별시 중구 다산로 248 (신당동, 신당파출소)"
4,서울청,서울중부,장충,파출소,서울특별시 중구 동호로 261
...,...,...,...,...,...
237,서울청,서울수서,대치,지구대,서울특별시 강남구 삼성로 203
238,서울청,서울수서,수서,파출소,서울특별시 강남구 밤고개로1길 48
239,서울청,서울수서,대왕,파출소,서울특별시 강남구 헌릉로 623
240,서울청,서울수서,일원,지구대,서울특별시 강남구 일원로3길 35 일원파출소


In [4]:
# 인덱스 수정 
police_seoul.reset_index(drop=False, inplace=True)

# 'index' 컬럼 drop
police_seoul = police_seoul.drop(['index'],axis='columns')
police_seoul

,지방청,경찰서,관서명,구분,주소
0,서울청,서울중부,을지,지구대,서울특별시 중구 을지로 234
1,서울청,서울중부,광희,지구대,서울특별시 중구 퇴계로 375-1
2,서울청,서울중부,약수,지구대,"서울특별시 중구 동호로 5길 15, 약수지구대"
3,서울청,서울중부,신당,파출소,"서울특별시 중구 다산로 248 (신당동, 신당파출소)"
4,서울청,서울중부,장충,파출소,서울특별시 중구 동호로 261
...,...,...,...,...,...
237,서울청,서울수서,대치,지구대,서울특별시 강남구 삼성로 203
238,서울청,서울수서,수서,파출소,서울특별시 강남구 밤고개로1길 48
239,서울청,서울수서,대왕,파출소,서울특별시 강남구 헌릉로 623
240,서울청,서울수서,일원,지구대,서울특별시 강남구 일원로3길 35 일원파출소


# 위경도 매핑(카카오API)

In [44]:
# 카카오API를 사용하여 주소->좌표 변환
import requests, json

def get_location(address):
    url = 'https://dapi.kakao.com/v2/local/search/address.json?query=' + address
    # 'KaKaoAK '는 그대로 두시고 개인키만 지우고 입력해 주세요.
    # ex) KakaoAK 6af8d4826f0e56c54bc794fa8a294
    headers = {"Authorization": "KakaoAK 7924f89c7d68dc7cfd158550d96fd730"}
    api_json = json.loads(str(requests.get(url,headers=headers).text))
    address = api_json['documents'][0]['address']
    crd = {"lat": str(address['y']), "lng": str(address['x'])}
    address_name = address['address_name']

    return crd

In [45]:
latitude = []
longitude =[]

In [46]:
#####주소를 위,경도 값으로 변환하기 #####

address = police_seoul.주소
for i in address[:len(address)]:
    try:
        latitude.append(get_location(i)['lat'])
        longitude.append(get_location(i)['lng'])
    except IndexError:
        latitude.append(np.nan)
        longitude.append(np.nan)

In [47]:
latitude

['37.5664750388987',
 '37.5652684538307',
 '37.5521442775352',
 '37.5650442671787',
 '37.5587633776035',
 '37.5608773647515',
 '37.5661815238113',
 '37.5705022157858',
 '37.5689505293286',
 '37.6000216905431',
 '37.6052486740828',
 '37.5814096566494',
 '37.576770888086',
 '37.5842278806099',
 '37.5814734297903',
 nan,
 '37.5689472465748',
 '37.5707552628269',
 '37.5760686673829',
 '37.5725887471851',
 '37.5680079853085',
 '37.5568085815143',
 '37.5635895671109',
 '37.5590136645085',
 '37.5578893434364',
 '37.558093937192',
 '37.5638008161255',
 '37.5698843736468',
 '37.5585525829582',
 '37.5952477134009',
 '37.5702755077088',
 '37.5760408583713',
 '37.582993055067',
 '37.5881707286561',
 '37.5836053290869',
 '37.5861083962149',
 '37.578611713366',
 '37.5892784329492',
 '37.5784936918859',
 '37.5726963187211',
 '37.5759851806728',
 '37.5707367126759',
 '37.5729430706406',
 '37.5389037956572',
 '37.549159039208',
 '37.5343115810699',
 '37.5310848781248',
 '37.5265772465655',
 '37.5361101

In [48]:
formatted_latitude = pd.DataFrame(latitude)
formatted_longitude = pd.DataFrame(longitude)

In [49]:
isna_index = formatted_latitude[formatted_latitude.isna().any(axis=1)].index

In [50]:
for index in isna_index:
    print(index, address.iloc[index])

15 서울특별시 종로구  송월길 167(교북동 10-38)
62 서울특별시 동대문구 한천로20길 7
78 서울특별시 영등포구 신길동 4533-16
84 서울특별시 영등포구 경인로102길 13
155 서울특별시 강서구  화곡로 310
208 서울특별시 송파구  토성로 64
232 서울특별시 도봉구  도봉로169길 43 도봉1파출소


In [56]:
formatted_latitude.iloc[78] = 37.4990668
formatted_longitude.iloc[78] = 126.912722

In [57]:
police_seoul['위도'] = formatted_latitude
police_seoul['경도'] = formatted_longitude

In [58]:
police_seoul

,지방청,경찰서,관서명,구분,주소,위도,경도
0,서울청,서울중부,을지,지구대,서울특별시 중구 을지로 234,37.5664750388987,127.003942069544
1,서울청,서울중부,광희,지구대,서울특별시 중구 퇴계로 375-1,37.5652684538307,127.013376356837
2,서울청,서울중부,약수,지구대,"서울특별시 중구 동호로 5길 15, 약수지구대",37.5521442775352,127.0123820902
3,서울청,서울중부,신당,파출소,"서울특별시 중구 다산로 248 (신당동, 신당파출소)",37.5650442671787,127.01638496365
4,서울청,서울중부,장충,파출소,서울특별시 중구 동호로 261,37.5587633776035,127.004802045236
...,...,...,...,...,...,...,...
237,서울청,서울수서,대치,지구대,서울특별시 강남구 삼성로 203,37.494873946034,127.062617624125
238,서울청,서울수서,수서,파출소,서울특별시 강남구 밤고개로1길 48,37.489127606674,127.104813214036
239,서울청,서울수서,대왕,파출소,서울특별시 강남구 헌릉로 623,37.4651760424395,127.10736243987
240,서울청,서울수서,일원,지구대,서울특별시 강남구 일원로3길 35 일원파출소,37.4924161491256,127.083373556556


In [62]:
police_seoul_drop = police_seoul.dropna()
police_seoul_drop

,지방청,경찰서,관서명,구분,주소,위도,경도
0,서울청,서울중부,을지,지구대,서울특별시 중구 을지로 234,37.5664750388987,127.003942069544
1,서울청,서울중부,광희,지구대,서울특별시 중구 퇴계로 375-1,37.5652684538307,127.013376356837
2,서울청,서울중부,약수,지구대,"서울특별시 중구 동호로 5길 15, 약수지구대",37.5521442775352,127.0123820902
3,서울청,서울중부,신당,파출소,"서울특별시 중구 다산로 248 (신당동, 신당파출소)",37.5650442671787,127.01638496365
4,서울청,서울중부,장충,파출소,서울특별시 중구 동호로 261,37.5587633776035,127.004802045236
...,...,...,...,...,...,...,...
237,서울청,서울수서,대치,지구대,서울특별시 강남구 삼성로 203,37.494873946034,127.062617624125
238,서울청,서울수서,수서,파출소,서울특별시 강남구 밤고개로1길 48,37.489127606674,127.104813214036
239,서울청,서울수서,대왕,파출소,서울특별시 강남구 헌릉로 623,37.4651760424395,127.10736243987
240,서울청,서울수서,일원,지구대,서울특별시 강남구 일원로3길 35 일원파출소,37.4924161491256,127.083373556556


In [64]:
# 저장 후 주석처리
# police_seoul_drop.to_csv("C:/sooeun/DATAexam/ConvergenceProject-School_Violence/Data/경찰청_2021_서울_경찰관서위치정보.csv") 